In [40]:
import tensorflow as tf
import numpy as np
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
 

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Dropout, InputLayer
from tensorflow.keras import optimizers, models
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.models import Sequential
from keras import optimizers
from keras.applications.mobilenet_v2 import preprocess_input


import torch 
from torchvision import transforms



In [41]:
base_model = MobileNetV2(include_top = False, input_shape=(200, 200, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = tf.nn.relu)(x)
preds = Dense(1, activation = tf.nn.sigmoid)(x)

model= Model(inputs=base_model.input,outputs=preds)

model.summary()

WARNING - `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.


Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 201, 201, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 100, 100, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 100, 100, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [42]:
epochs = 5
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.8
opt = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [5]:
model_0 = model

model_0.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
X_test = np.load('cnn/test/patches_2.npy')
y_true = np.load('cnn/test/labels_2.npy')
y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes = model_0.predict(X_test)
y_pred = classes.round()
print('Accuracy ', accuracy_score(y_true, y_pred))

Accuracy  0.4152046783625731


In [7]:
X_test_1 = np.load('cnn/test/patches.npy')
y_true_1 = np.load('cnn/test/labels.npy')

X_test_2 = np.load('cnn/test/patches_2.npy')
y_true_2 = np.load('cnn/test/labels_2.npy')

X_test = np.concatenate((X_test_1, X_test_2), axis = 0)
y_true = np.concatenate((y_true_1, y_true_2), axis = 0)

print(X_test.shape)
print(y_true.shape)

y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes = model_0.predict(X_test)
y_pred = classes.round()
print('Accuracy ', accuracy_score(y_true, y_pred))

(276, 200, 200, 3)
(276,)
Accuracy  0.391304347826087


## CNN trained on Handlabeled

In [90]:
patches_hand_pos = np.load('cnn/train_hand/patches_pos.npy')
patches_hand_neg = np.load('cnn/train_hand/patches_neg.npy')

In [8]:
patches_hand_pos = patches_hand_pos[np.random.choice(len(patches_hand_pos), size = 100, replace = False)]
patches_hand_neg = patches_hand_neg[np.random.choice(len(patches_hand_neg), size = 100, replace = False)]

In [91]:
n_pos = len(patches_hand_pos)
n_neg = len(patches_hand_neg)
n_pos, n_neg

(163, 237)

In [92]:
y_pos = np.array([0]*n_pos)
y_neg = np.array([1]*n_neg)

X = np.concatenate((patches_hand_pos, patches_hand_neg), axis=0)
y = np.concatenate((y_pos, y_neg), axis=0)

randomize = np.arange(len(X))
np.random.shuffle(randomize)
X = X[randomize]
y = y[randomize]

print(X.shape)
print(y.shape)

(400, 200, 200, 3)
(400,)


In [93]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 2)

In [94]:
#CNN model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import preprocess_input

# All images will be rescaled by 1./255
train_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_musacea.flow( X_train, y_train, # All images will be resized to 200x200
        batch_size=128)

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_musacea.flow(X_val, y_val,  # All images will be resized to 200x200
        batch_size=128,
        shuffle=False)

In [95]:
model_hand = model

model_hand.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
history_hand = model_hand.fit(train_generator,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator)
      #use_multiprocessing=True, workers=4)

Epoch 1/5
3/3 [==============================] - 4s 1s/step - loss: 0.4378 - accuracy: 0.8813 - val_loss: 0.0627 - val_accuracy: 0.9875
Epoch 2/5
3/3 [==============================] - 3s 923ms/step - loss: 0.0883 - accuracy: 0.9625 - val_loss: 0.0617 - val_accuracy: 0.9875
Epoch 3/5
3/3 [==============================] - 3s 928ms/step - loss: 0.0192 - accuracy: 0.9969 - val_loss: 0.1041 - val_accuracy: 0.9750
Epoch 4/5
3/3 [==============================] - 3s 936ms/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 0.1329 - val_accuracy: 0.9625
Epoch 5/5
3/3 [==============================] - 3s 979ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.1465 - val_accuracy: 0.9625


WARNING - Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2609s vs `on_train_batch_end` time: 0.8408s). Check your callbacks.


In [96]:
X_test = np.load('cnn/test/patches_2.npy')
y_true = np.load('cnn/test/labels_2.npy')


y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_hand = model_hand.predict(X_test)
y_pred = classes_hand.round()
print('Accuracy ', accuracy_score(y_true, y_pred))

Accuracy  0.5672514619883041


In [97]:
X_test_1 = np.load('cnn/test/patches.npy')
y_true_1 = np.load('cnn/test/labels.npy')

X_test_2 = np.load('cnn/test/patches_2.npy')
y_true_2 = np.load('cnn/test/labels_2.npy')

X_test = np.concatenate((X_test_1, X_test_2), axis = 0)
y_true = np.concatenate((y_true_1, y_true_2), axis = 0)

print(X_test.shape)
print(y_true.shape)

y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_hand = model_hand.predict(X_test)
y_pred = classes_hand.round()
print('Accuracy ', accuracy_score(y_true, y_pred))


(276, 200, 200, 3)
(276,)
Accuracy  0.6702898550724637


## CNN trained on NN

In [51]:
patches_nn_pos = np.load('cnn/train_nn/patches_pos.npy')
patches_nn_neg = np.load('cnn/train_nn/patches_neg.npy')

In [143]:
#Rebalance
n_min = min(len(patches_nn_pos), len(patches_nn_neg))

patches_pos = patches_nn_pos[np.random.choice(len(patches_nn_pos), size=n_min, replace=False)]
patches_neg = patches_nn_neg[np.random.choice(len(patches_nn_neg), size=n_min, replace=False)]

In [61]:
# Initial
patches_pos = patches_nn_pos
patches_neg = patches_nn_neg

In [144]:
# Choose to join handlabeled training set 
patches_pos = np.concatenate((patches_pos, patches_hand_pos), axis=0)
patches_neg = np.concatenate((patches_neg, patches_hand_neg), axis=0)

In [145]:
n_pos = len(patches_pos)
n_neg = len(patches_neg)
n_pos, n_neg

(1666, 1740)

In [146]:
y_pos = np.array([0]*n_pos)
y_neg = np.array([1]*n_neg)

X = np.concatenate((patches_pos, patches_neg), axis=0)
y = np.concatenate((y_pos, y_neg), axis=0)

randomize = np.arange(len(X))
np.random.shuffle(randomize)
X = X[randomize]
y = y[randomize]

print(X.shape)
print(y.shape)

(3406, 200, 200, 3)
(3406,)


In [147]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 2)

In [148]:
#CNN model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_musacea.flow( X_train, y_train, # All images will be resized to 200x200
        batch_size=128)

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_musacea.flow(X_val, y_val,  # All images will be resized to 200x200
        batch_size=128,
        shuffle=False)

In [149]:
model_nn = model

model_nn.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
history_nn = model_nn.fit(train_generator,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator)
      #use_multiprocessing=True, workers=8)

Epoch 1/5
22/22 [==============================] - 27s 1s/step - loss: 0.5687 - accuracy: 0.7228 - val_loss: 0.7082 - val_accuracy: 0.6232
Epoch 2/5
22/22 [==============================] - 25s 1s/step - loss: 0.3553 - accuracy: 0.8322 - val_loss: 0.7969 - val_accuracy: 0.5909
Epoch 3/5
22/22 [==============================] - 25s 1s/step - loss: 0.2997 - accuracy: 0.8634 - val_loss: 0.8423 - val_accuracy: 0.5777
Epoch 4/5
22/22 [==============================] - 26s 1s/step - loss: 0.2650 - accuracy: 0.8733 - val_loss: 0.8916 - val_accuracy: 0.5777
Epoch 5/5
22/22 [==============================] - 26s 1s/step - loss: 0.2506 - accuracy: 0.8796 - val_loss: 0.9446 - val_accuracy: 0.5660


In [150]:
X_test = np.load('cnn/test/patches_2.npy')
y_true = np.load('cnn/test/labels_2.npy')
y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_nn = model_nn.predict(X_test)
y_pred = classes_nn.round()
print('Accuracy ', accuracy_score(y_true, y_pred))


Accuracy  0.4093567251461988


In [152]:
X_test_1 = np.load('cnn/test/patches.npy')
y_true_1 = np.load('cnn/test/labels.npy')

X_test_2 = np.load('cnn/test/patches_2.npy')
y_true_2 = np.load('cnn/test/labels_2.npy')

X_test = np.concatenate((X_test_1, X_test_2), axis = 0)
y_true = np.concatenate((y_true_1, y_true_2), axis = 0)

print(X_test.shape)
print(y_true.shape)

y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_nn = model_nn.predict(X_test)
y_pred_nn = classes_nn.round()
print('Accuracy ', accuracy_score(y_true, y_pred_nn))

(276, 200, 200, 3)
(276,)
Accuracy  0.5434782608695652


## CNN trained on OT (Non-Greedy)

In [31]:
patches_ot_pos = np.load('cnn/train_ot/patches_pos.npy')
patches_ot_neg = np.load('cnn/train_ot/patches_neg.npy')

In [153]:
#Rebalance
n_min = min(len(patches_ot_pos), len(patches_ot_neg))

patches_pos = patches_ot_pos[np.random.choice(len(patches_ot_pos), size=n_min, replace=False)]
patches_neg = patches_ot_neg[np.random.choice(len(patches_ot_neg), size=n_min, replace=False)]

In [155]:
# Initial
patches_pos = patches_ot_pos
patches_neg = patches_ot_neg

In [156]:
# Choose to join handlabeled training set 
patches_pos = np.concatenate((patches_pos, patches_hand_pos), axis=0)
patches_neg = np.concatenate((patches_neg, patches_hand_neg), axis=0)

In [162]:
n_pos = len(patches_pos)
n_neg = len(patches_neg)
n_pos, n_neg

(1667, 3398)

In [163]:
y_pos = np.array([0]*n_pos)
y_neg = np.array([1]*n_neg)

X = np.concatenate((patches_pos, patches_neg), axis=0)
y = np.concatenate((y_pos, y_neg), axis=0)

randomize = np.arange(len(X))
np.random.shuffle(randomize)
X = X[randomize]
y = y[randomize]

print(X.shape)
print(y.shape)

(5065, 200, 200, 3)
(5065,)


In [164]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 2)

In [165]:
#CNN model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be preprocessed to be compatible to vgg16 input data
train_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_musacea.flow( X_train, y_train, # All images will be resized to 200x200
        batch_size=128)

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_musacea.flow(X_val, y_val,  # All images will be resized to 200x200
        batch_size=128,
        shuffle=False)

In [166]:
model_ot = model


model_ot.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

history_ot = model_ot.fit(train_generator,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator)
      #,use_multiprocessing=True, workers=8)

Epoch 1/5
32/32 [==============================] - 39s 1s/step - loss: 0.3068 - accuracy: 0.8458 - val_loss: 0.5208 - val_accuracy: 0.7502
Epoch 2/5
32/32 [==============================] - 38s 1s/step - loss: 0.2635 - accuracy: 0.8630 - val_loss: 0.5256 - val_accuracy: 0.7532
Epoch 3/5
32/32 [==============================] - 38s 1s/step - loss: 0.2477 - accuracy: 0.8650 - val_loss: 0.5531 - val_accuracy: 0.7374
Epoch 4/5
32/32 [==============================] - 38s 1s/step - loss: 0.2409 - accuracy: 0.8680 - val_loss: 0.5656 - val_accuracy: 0.7315
Epoch 5/5
32/32 [==============================] - 38s 1s/step - loss: 0.2330 - accuracy: 0.8719 - val_loss: 0.5677 - val_accuracy: 0.7374


WARNING - Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2669s vs `on_train_batch_end` time: 0.8787s). Check your callbacks.


In [167]:
X_test = np.load('cnn/test/patches_2.npy')
y_true = np.load('cnn/test/labels_2.npy')
y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_ot = model_ot.predict(X_test)
y_pred_ot = classes_ot.round()
print('Accuracy ', accuracy_score(y_true, y_pred_ot))

Accuracy  0.52046783625731


In [168]:
X_test_1 = np.load('cnn/test/patches.npy')
y_true_1 = np.load('cnn/test/labels.npy')

X_test_2 = np.load('cnn/test/patches_2.npy')
y_true_2 = np.load('cnn/test/labels_2.npy')

X_test = np.concatenate((X_test_1, X_test_2), axis = 0)
y_true = np.concatenate((y_true_1, y_true_2), axis = 0)

print(X_test.shape)
print(y_true.shape)

y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_ot = model_ot.predict(X_test)
y_pred_ot = classes_ot.round()
print('Accuracy ', accuracy_score(y_true, y_pred_ot))

(276, 200, 200, 3)
(276,)
Accuracy  0.5833333333333334


## CNN trained on OT with CNN features

In [22]:
patches_ot_cnn_pos = np.load('cnn/train_ot_cnn/patches_pos.npy')
patches_ot_cnn_neg = np.load('cnn/train_ot_cnn/patches_neg.npy')

In [133]:
#Rebalance
n_min = min(len(patches_ot_pos), len(patches_ot_neg))

patches_pos = patches_ot_cnn_pos[np.random.choice(len(patches_ot_cnn_pos), size=n_min, replace=False)]
patches_neg = patches_ot_cnn_neg[np.random.choice(len(patches_ot_cnn_neg), size=n_min, replace=False)]

In [81]:
# Initial
patches_pos = patches_ot_cnn_pos
patches_neg = patches_ot_cnn_neg

In [125]:
# Choose to join handlabeled training set 
patches_pos = np.concatenate((patches_pos, patches_hand_pos), axis=0)
patches_neg = np.concatenate((patches_neg, patches_hand_neg), axis=0)

In [126]:
n_pos = len(patches_pos)
n_neg = len(patches_neg)
n_pos, n_neg

(1667, 1741)

In [127]:
y_pos = np.array([0]*n_pos)
y_neg = np.array([1]*n_neg)

X = np.concatenate((patches_pos, patches_neg), axis=0)
y = np.concatenate((y_pos, y_neg), axis=0)

randomize = np.arange(len(X))
np.random.shuffle(randomize)
X = X[randomize]
y = y[randomize]

print(X.shape)
print(y.shape)

(3408, 200, 200, 3)
(3408,)


In [128]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 2)

In [129]:
#CNN model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_musacea = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow training images in batches of 120 using train_datagen generator
train_generator = train_musacea.flow( X_train, y_train, # All images will be resized to 200x200
        batch_size=128)

# Flow validation images in batches of 19 using valid_datagen generator
validation_generator = validation_musacea.flow(X_val, y_val,  # All images will be resized to 200x200
        batch_size=128,
        shuffle=False)

In [130]:
model_ot_cnn = model
model_ot_cnn.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
history_ot_cnn = model_ot_cnn.fit(train_generator,  
      epochs=5,
      verbose=1,
      validation_data = validation_generator)
      #use_multiprocessing=True, workers=12)

Epoch 1/5
22/22 [==============================] - 28s 1s/step - loss: 0.0984 - accuracy: 0.9688 - val_loss: 0.3731 - val_accuracy: 0.8534
Epoch 2/5
22/22 [==============================] - 25s 1s/step - loss: 0.0455 - accuracy: 0.9853 - val_loss: 0.3554 - val_accuracy: 0.8651
Epoch 3/5
22/22 [==============================] - 26s 1s/step - loss: 0.0347 - accuracy: 0.9901 - val_loss: 0.3823 - val_accuracy: 0.8519
Epoch 4/5
22/22 [==============================] - 26s 1s/step - loss: 0.0250 - accuracy: 0.9934 - val_loss: 0.3895 - val_accuracy: 0.8622
Epoch 5/5
22/22 [==============================] - 26s 1s/step - loss: 0.0225 - accuracy: 0.9934 - val_loss: 0.3977 - val_accuracy: 0.8578


WARNING - Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2618s vs `on_train_batch_end` time: 0.8581s). Check your callbacks.


In [131]:
X_test = np.load('cnn/test/patches_2.npy')
y_true = np.load('cnn/test/labels_2.npy')
print(X_test.shape)
y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_ot_cnn = model_ot_cnn.predict(X_test)
y_pred_ot_cnn = classes_ot_cnn.round()
print('Accuracy ', accuracy_score(y_true, y_pred_ot_cnn))

(171, 200, 200, 3)
Accuracy  0.7309941520467836


In [132]:
X_test_1 = np.load('cnn/test/patches.npy')
y_true_1 = np.load('cnn/test/labels.npy')

X_test_2 = np.load('cnn/test/patches_2.npy')
y_true_2 = np.load('cnn/test/labels_2.npy')

X_test = np.concatenate((X_test_1, X_test_2), axis = 0)
y_true = np.concatenate((y_true_1, y_true_2), axis = 0)

print(X_test.shape)
print(y_true.shape)

y_true = y_true.astype(int)
X_test = preprocess_input(X_test)
classes_ot_cnn = model_ot_cnn.predict(X_test)
y_pred_ot_cnn = classes_ot_cnn.round()
print('Accuracy ', accuracy_score(y_true, y_pred_ot_cnn))

(276, 200, 200, 3)
(276,)
Accuracy  0.8007246376811594
